In [2]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
import re
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Morsi Store
[nltk_data]     DZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Morsi Store
[nltk_data]     DZ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Morsi Store
[nltk_data]     DZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def preprocess_text(text):

    def is_english_word(word):
        # synsets = synonyms sets
        synsets = wordnet.synsets(word)
        # check if the given word is equal to the noun of lemmatized version of synonyms sets
        return len(synsets) > 0 and synsets[0].lemmas()[0].name().lower() == word.lower()

    # remove non-word characters and numbers using regular expressions
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'<[^>]+>', '', text)

    # tokenize the text
    tokens = word_tokenize(text.lower())

    # remove English stop words
    english_stopwords = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in english_stopwords]
    
    # filter out non-English words
    english_words = [token for token in filtered_tokens if is_english_word(token)]
    
    return english_words

In [ ]:
lemmatizer = WordNetLemmatizer
train_light = json.loads(open('train_light.json').read())

words = []
classes = []
documents = []

train_light.columns

NameError: name 'WordNetLemmatizer' is not defined